In [ ]:

!pip install -U transformers #--upgrade

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline #AutoModelForSeq2SeqLM for t5

from transformers.utils import logging

In [ ]:
#!pip install -U accelerate #for GPU use in CPU for HF
#HF API call to model
!pip install transformers_stream_generator
!pip install tiktoken
!pip install -U tokenizers

In [3]:
import json

def read_json_objects(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            try:
                yield json.loads(line)
            except json.JSONDecodeError:
                continue

# Example usage
file_path = '/content/en_fact.json'
#for obj in read_json_objects(file_path):
#    print(obj)

In [4]:
import json

def read_json_objects(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            try:
                yield json.loads(line)
            except json.JSONDecodeError:
                continue

# Example usage
file_path = '/content/en_refine.json'
#for obj in read_json_objects(file_path):
#    print(obj)

In [5]:
import json
import csv

def read_json_objects(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            try:
                yield json.loads(line)
            except json.JSONDecodeError:
                continue

def json_to_csv(json_file_path, csv_file_path):
    # Open the CSV file in write mode
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        # Create a CSV DictWriter object
        writer = None
        for obj in read_json_objects(json_file_path):
            # Write headers only once
            if writer is None:
                headers = obj.keys()  # Get the keys from the first object as headers
                writer = csv.DictWriter(csvfile, fieldnames=headers)
                writer.writeheader()

            # Write the object data as a row in the CSV
            writer.writerow(obj)

# Example usage
json_file_path = '/content/en_fact.json'
csv_file_path = '/content/outputfact.csv'
json_to_csv(json_file_path, csv_file_path)

In [6]:
import json
import csv

def read_json_objects(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            try:
                yield json.loads(line)
            except json.JSONDecodeError:
                continue

def json_to_csv(json_file_path, csv_file_path):
    # Open the CSV file in write mode
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        # Create a CSV DictWriter object
        writer = None
        for obj in read_json_objects(json_file_path):
            # Write headers only once
            if writer is None:
                headers = obj.keys()  # Get the keys from the first object as headers
                writer = csv.DictWriter(csvfile, fieldnames=headers)
                writer.writeheader()

            # Write the object data as a row in the CSV
            writer.writerow(obj)

# Example usage
json_file_path = '/content/en_refine.json'
csv_file_path = '/content/outputrefine.csv'
json_to_csv(json_file_path, csv_file_path)


In [43]:
import csv
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from random import shuffle
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define the load_model function
def load_model(model_name):
    """
    Loads a Hugging Face model and tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return model, tokenizer

# Load multiple models into a dictionary
models = {
    "BLOOM": load_model('bigscience/bloom-560m'),  # Ensure this returns a model instance
    "MiniLLM": SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')  # This should be correct
}
def get_embedding(text):
    """
    Generate an embedding for the given text using Hugging Face's Sentence-Transformer model.
    """
    return model.encode(text)

def add_noise(text, noise_level=0.1):
    """
    Adds noise to the text by removing a random selection of words.
    Noise level defines the percentage of words to remove.
    """
    words = text.split()
    num_words_to_remove = int(len(words) * noise_level)
    shuffle(words)  # Shuffle to remove random words
    noisy_text = ' '.join(words[:-num_words_to_remove]) if num_words_to_remove > 0 else text
    #print(noisy_text)
    return noisy_text

In [44]:
def calculate_noise_robustness_eucdist(original_text, model_name):
    """
    Calculate noise robustness using Euclidean distance between original text embedding
    and noisy text embedding.

    Parameters:
    - original_text (str): The original text to evaluate.
    - model_name (str): The name of the model to use for generating embeddings.

    Returns:
    - float: The Euclidean distance between the original and noisy text embeddings.
    """
    try:
        # Get the embedding function based on the model name
        embedding_fn = models[model_name]  # Assuming models is a dictionary of loaded models

        # Check if embedding_fn is indeed callable
        if isinstance(embedding_fn, tuple):
            embedding_fn = embedding_fn[0]  # Adjust this if your model is stored as a tuple

        # Get the embedding for the original text
        original_embedding = embedding_fn.encode(original_text)

        # Add noise to the original text
        noisy_text = add_noise(original_text)

        # Get the embedding for the noisy text
        noisy_embedding = embedding_fn.encode(noisy_text)

        # Compute Euclidean distance between original and noisy embeddings
        distance = euclidean(original_embedding, noisy_embedding)

        return distance

    except Exception as e:
        print(f"Error calculating noise robustness: {e}")
        return None  # Return None or handle error as needed


In [ ]:
import csv
import pandas as pd
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import euclidean
from transformers import pipeline  #Import pipeline from transformers

# Load the SentenceTransformer model (used as an output encoding step, and MUST exist.)
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')  # Always use a sentence transformer

def generate_text(model, query):
    """
    Generate text output from a given model based on the query.
    """
    try:
        generator = pipeline('text-generation', model=model, tokenizer=model.tokenizer) # Model MUST have tokenizer
        generated_text = generator(query, max_length=50, num_return_sequences=1)[0]['generated_text']  # Adjust max_length
        return generated_text
    except Exception as e:
        print(f"Error generating text: {e}")
        return None

def add_noise(text, noise_level=0.1):
    """Adds noise by removing a random selection of words."""
    words = text.split()
    num_words_to_remove = int(len(words) * noise_level)
    import random
    random.shuffle(words)
    noisy_text = ' '.join(words[:-num_words_to_remove]) if num_words_to_remove > 0 else text
    return noisy_text

def calculate_noise_robustness_eucdist(original_text, model_name):
    """
    Calculates noise robustness by generating text, embedding, and comparing distances.
    """
    try:
        # Get the model from the models dictionary
        model = models[model_name] # model is now the raw LLM object

        # Generate text from the original query
        generated_text = generate_text(model, original_text)  # Generate with the original

        if generated_text is None:
            return None  # Handle generation failure

        # Add noise to the original text and generate text again
        noisy_text = add_noise(original_text)
        noisy_generated_text = generate_text(model, noisy_text)  # Generate with the noisy query

        if noisy_generated_text is None:
            return None  # Handle generation failure


        # Embed both generated texts using the sentence transformer
        original_embedding = sentence_model.encode(generated_text) #Now use sentence_model
        noisy_embedding = sentence_model.encode(noisy_generated_text) #Now use sentence_model


        # Calculate Euclidean distance
        distance = euclidean(original_embedding, noisy_embedding)
        return distance

    except Exception as e:
        print(f"Error calculating noise robustness: {e}")
        return None

#Refactored to allow better handling of the models themselves
def load_model(model_string):
  from transformers import AutoModelForCausalLM, AutoTokenizer
  model = AutoModelForCausalLM.from_pretrained(model_string)
  tokenizer = AutoTokenizer.from_pretrained(model_string)
  model.tokenizer = tokenizer #Important
  return model #Now returns the transformer model

# Define models (MUST BE DEFINED BEFORE CALLING THE PROCESS FUNCTION)
models = {
    "BLOOM": load_model('bigscience/bloom-560m'), #This should now work, but verify your memory constraints
    "MiniLLM": load_model('sentence-transformers/all-MiniLM-L6-v2') #Not actually MiniLLM, and will cause problems for text generation.
}

def process_csv_and_calculate_metrics(csv_file_path):
    """
    Processes CSV, calculates noise robustness using each specified model.
    """
    results = []

    try:
        with open(csv_file_path, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)

            for row in reader:
                try:
                    query = row['query']
                    answer = row['answer']
                    positive = row.get('positive', None)
                    negative = row.get('negative', None)

                    metrics_row = {
                        'query': query,
                        'answer': answer,
                        'positive': positive,
                        'negative': negative
                    }

                    for model_name in models.keys():
                        try:
                            noise_robustness_eucdist = calculate_noise_robustness_eucdist(query, model_name)
                            metrics_row[f'noise_robustness_eucdist_{model_name}'] = noise_robustness_eucdist
                        except Exception as model_err:
                            print(f"Error with model {model_name}: {model_err}")
                            metrics_row[f'noise_robustness_eucdist_{model_name}'] = None # Handle model-specific errors

                    results.append(metrics_row)

                except Exception as row_err:
                    print(f"Error processing row: {row_err}")

    except FileNotFoundError:
        print(f"Error: CSV file not found at {csv_file_path}")
        return pd.DataFrame()
    except Exception as file_err:
        print(f"Error reading CSV file: {file_err}")
        return pd.DataFrame()

    results_df = pd.DataFrame(results)
    return results_df

# Example usage
csv_file_path = '/content/outputrefine.csv'  # Update with the actual CSV file path
metrics_df = process_csv_and_calculate_metrics(csv_file_path)

# Print or save the DataFrame as needed
print(metrics_df)


If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Device set to use cpu
Device set to use 

In [45]:
def process_csv_and_calculate_metrics(csv_file_path):
    """
    Read the CSV file, calculate noise robustness and information integration
    for each row, and return the results.
    """
    results = []

    try:
        with open(csv_file_path, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)

            for row in reader:
                try:  # Inner try-except for row-specific errors
                    query = row['query']
                    answer = row['answer']
                    positive = row.get('positive', None)  # Use get() to avoid KeyError
                    negative = row.get('negative', None)  # Use get() to avoid KeyError

                    # Initialize a dictionary to hold metrics for this row
                    metrics_row = {
                        'query': query,
                        'answer': answer,
                        'positive': positive,
                        'negative': negative
                    }

                    # Iterate through each model to calculate metrics
                    for model_name in models.keys():
                        try:
                            # Calculate noise robustness using Euclidean distance for the specified model
                            noise_robustness_eucdist = calculate_noise_robustness_eucdist(query, model_name)

                            # Store the result in the metrics_row dictionary
                            metrics_row[f'noise_robustness_eucdist_{model_name}'] = noise_robustness_eucdist
                        except Exception as model_err:
                            print(f"Error with model {model_name}: {model_err}")
                            metrics_row[f'noise_robustness_eucdist_{model_name}'] = None #Or some default value

                    # Append metrics_row to results list
                    results.append(metrics_row)

                except Exception as row_err:
                    print(f"Error processing row: {row_err}") #Handles the error of a single row
                    #Potentially add problematic row information

    except FileNotFoundError:
        print(f"Error: CSV file not found at {csv_file_path}")
        return pd.DataFrame() #Return empty dataframe to avoid later errors
    except Exception as file_err:
        print(f"Error reading CSV file: {file_err}")
        return pd.DataFrame() #Return empty dataframe

    # Convert results to a DataFrame for better handling (optional)
    results_df = pd.DataFrame(results)

    return results_df


# Example usage
csv_file_path = '/content/outputrefine.csv'  # Update with the actual CSV file path
metrics_df = process_csv_and_calculate_metrics(csv_file_path)

# Print or save the DataFrame as needed
print(metrics_df)


Error calculating noise robustness: 'BloomForCausalLM' object has no attribute 'encode'
Error calculating noise robustness: 'BloomForCausalLM' object has no attribute 'encode'
Error calculating noise robustness: 'BloomForCausalLM' object has no attribute 'encode'
Error calculating noise robustness: 'BloomForCausalLM' object has no attribute 'encode'
Error calculating noise robustness: 'BloomForCausalLM' object has no attribute 'encode'
Error calculating noise robustness: 'BloomForCausalLM' object has no attribute 'encode'
Error calculating noise robustness: 'BloomForCausalLM' object has no attribute 'encode'
Error calculating noise robustness: 'BloomForCausalLM' object has no attribute 'encode'
Error calculating noise robustness: 'BloomForCausalLM' object has no attribute 'encode'
Error calculating noise robustness: 'BloomForCausalLM' object has no attribute 'encode'
Error calculating noise robustness: 'BloomForCausalLM' object has no attribute 'encode'
Error calculating noise robustne

KeyboardInterrupt: 

In [40]:
def calculate_noise_robustness_cosinesim(original_text, embedding_fn):
    """
    Calculate the noise robustness by comparing the cosine similarity of
    the original text embedding and the noisy text embedding.
    """
    # Get the embedding for the original and noisy texts
    original_embedding = embedding_fn(original_text)
    noisy_text = add_noise(original_text)  # Add noise to the original text
    noisy_embedding = embedding_fn(noisy_text)

    # Compute cosine similarity between original and noisy embeddings
    similarity = cosine_similarity([original_embedding], [noisy_embedding])[0][0]
    return similarity


from scipy.spatial.distance import euclidean

def calculate_noise_robustness_eucdistold(original_text, embedding_fn):
    original_embedding = embedding_fn(original_text)
    noisy_text = add_noise(original_text)
    noisy_embedding = embedding_fn(noisy_text)

    # Compute Euclidean distance between original and noisy embeddings
    distance = euclidean(original_embedding, noisy_embedding)
    return distance

In [26]:
def process_csv_and_calculate_metrics(csv_file_path):
    """
    Read the CSV file, calculate noise robustness and information integration
    for each row, and return the results.
    """
    results = []
    with open(csv_file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)

        for row in reader:
            query = row['query']
            answer = row['answer']
            #fakeanswer = row['fakeanswer']
            positive = row['positive']
            negative = row['negative']

            # Calculate Noise Robustness for the query
            noise_robustness_cosinesim = 0 #calculate_noise_robustness_cosinesim(query, get_embedding)
            noise_robustness_eucdist = calculate_noise_robustness_eucdist(query, get_embedding)


            results.append({'noise_robustness_eucdist': noise_robustness_eucdist})
            '''
            # Store the results in a list
            results.append({
                'id': row['id'],
                'query': query,
                'noise_robustness_consinesim': noise_robustness_cosinesim,
                'noise_robustness_eucdist': calculate_noise_robustness_eucdist
            })
            '''

    return results

# Example usage
csv_file_path = '/content/outputrefine.csv'  # Update with the actual CSV file path
metrics = process_csv_and_calculate_metrics(csv_file_path)


In [31]:
rdf = pd.DataFrame(metrics)
rdf

,noise_robustness_eucdist
0,0.560356
1,0.000000
2,0.000000
3,0.516986
4,0.000000
...,...
295,0.000000
296,0.000000
297,0.000000
298,0.000000


In [32]:
#average, min and max of rdf
average_value = rdf['noise_robustness_eucdist'].mean()
min_value = rdf['noise_robustness_eucdist'].min()
max_value = rdf['noise_robustness_eucdist'].max()

# Print results
print(f"Average: {average_value:.2f}")
print(f"Minimum: {min_value:.2f}")
print(f"Maximum: {max_value:.2f}")

Average: 0.15
Minimum: 0.00
Maximum: 0.75


In [ ]:
import csv
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from random import shuffle
import pandas as pd  # Import pandas to handle tabular data

# List of models for comparison
models = [
    ('all-MiniLM-L6-v2', SentenceTransformer('all-MiniLM-L6-v2')),
    ('paraphrase-MiniLM-L6-v2', SentenceTransformer('paraphrase-MiniLM-L6-v2')),
    ('distilbert-base-nli-stsb-mean-tokens', SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')),
    ('roberta-base-nli-stsb-mean-tokens', SentenceTransformer('roberta-base-nli-stsb-mean-tokens'))
]

def get_embedding(model, text):
    """
    Generate an embedding for the given text using the specified model.
    """
    return model.encode(text)

def add_noise(text, noise_level=0.1):
    """
    Adds noise to the text by removing a random selection of words.
    Noise level defines the percentage of words to remove.
    """
    words = text.split()
    num_words_to_remove = int(len(words) * noise_level)
    shuffle(words)  # Shuffle to remove random words
    noisy_text = ' '.join(words[:-num_words_to_remove]) if num_words_to_remove > 0 else text
    return noisy_text

def calculate_noise_robustness(original_text, model, embedding_fn):
    """
    Calculate the noise robustness by comparing the cosine similarity of
    the original text embedding and the noisy text embedding.
    """
    # Get the embedding for the original and noisy texts
    original_embedding = embedding_fn(model, original_text)
    noisy_text = add_noise(original_text)  # Add noise to the original text
    noisy_embedding = embedding_fn(model, noisy_text)

    # Compute cosine similarity between original and noisy embeddings
    similarity = cosine_similarity([original_embedding], [noisy_embedding])[0][0]
    return similarity

def calculate_information_integration(text1, text2, model, embedding_fn):
    """
    Calculate information integration by computing the cosine similarity
    between two text embeddings.
    """
    embedding1 = embedding_fn(model, text1)
    embedding2 = embedding_fn(model, text2)

    # Compute cosine similarity
    similarity = cosine_similarity([embedding1], [embedding2])[0][0]
    return similarity

def process_csv_and_calculate_metrics(csv_file_path):
    """
    Read the CSV file, calculate noise robustness and information integration
    for each row, and return the results.
    """
    all_results = []

    # Loop through each model for comparison
    for model_name, model in models:
        results = []
        with open(csv_file_path, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)

            for row in reader:
                query = row['query']
                answer = row['answer']
                positive = row['positive']
                negative = row['negative']

                # Calculate Noise Robustness for the query
                noise_robustness = calculate_noise_robustness(query, model, get_embedding)

                # Calculate Information Integration (compare query with answer, positive, negative)
                integration_answer = calculate_information_integration(query, answer, model, get_embedding)
                integration_positive = calculate_information_integration(query, positive, model, get_embedding)
                integration_negative = calculate_information_integration(query, negative, model, get_embedding)

                # Store the results in a list
                results.append({
                    'Model': model_name,
                    'Noise Robustness': noise_robustness,
                    'Information Integration': integration_answer,
                    'Integration Positive': integration_positive,
                    'Integration Negative': integration_negative,
                })

        # Convert the list of results into a pandas DataFrame
        df = pd.DataFrame(results)

        # Calculate averages for the specified columns
        averages = df[['Noise Robustness', 'Information Integration', 'Integration Positive', 'Integration Negative']].mean()

        # Add the averages as the last row
        averages_row = pd.DataFrame([averages], columns=averages.index)
        averages_row['Model'] = 'Average'

        # Append the averages row to the DataFrame
        df = pd.concat([df, averages_row], ignore_index=True)

        all_results.append(df)

    # Concatenate results from all models into a single DataFrame
    final_df = pd.concat(all_results, ignore_index=True)

    # Return the final DataFrame
    return final_df

# Example usage
csv_file_path = '/content/outputrefine.csv'  # Update with the actual CSV file path
metrics_df = process_csv_and_calculate_metrics(csv_file_path)

# Print the DataFrame (tabular format)
print(metrics_df)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#Find average
"""
Average Noise Robustness: 0.9631411
Average Information Integration (Answer): 0.25863132
Average Information Integration (Positive): 0.6417046
Average Information Integration (Negative): 0.5523686
Average Negative Rejection: 0.5523686
Average Counterfactual Robustness: nan
"""
import csv
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from random import shuffle

# Load the SentenceTransformer model from Hugging Face
model = SentenceTransformer('all-MiniLM-L6-v2')

def get_embedding(text):
    """
    Generate an embedding for the given text using Hugging Face's Sentence-Transformer model.
    """
    return model.encode(text)

def add_noise(text, noise_level=0.1):
    """
    Adds noise to the text by removing a random selection of words.
    Noise level defines the percentage of words to remove.
    """
    words = text.split()
    num_words_to_remove = int(len(words) * noise_level)
    shuffle(words)  # Shuffle to remove random words
    noisy_text = ' '.join(words[:-num_words_to_remove]) if num_words_to_remove > 0 else text
    return noisy_text

def calculate_noise_robustness(original_text, embedding_fn):
    """
    Calculate the noise robustness by comparing the cosine similarity of
    the original text embedding and the noisy text embedding.
    """
    # Get the embedding for the original and noisy texts
    original_embedding = embedding_fn(original_text)
    noisy_text = add_noise(original_text)  # Add noise to the original text
    noisy_embedding = embedding_fn(noisy_text)

    # Compute cosine similarity between original and noisy embeddings
    similarity = cosine_similarity([original_embedding], [noisy_embedding])[0][0]
    return similarity

def calculate_information_integration(text1, text2, embedding_fn):
    """
    Calculate information integration by computing the cosine similarity
    between two text embeddings.
    """
    embedding1 = embedding_fn(text1)
    embedding2 = embedding_fn(text2)

    # Compute cosine similarity
    similarity = cosine_similarity([embedding1], [embedding2])[0][0]
    return similarity

def calculate_negative_rejection(query, negative, embedding_fn):
    """
    Calculate negative rejection by measuring the cosine similarity between
    the query and the negative text. A lower similarity means better rejection.
    """
    query_embedding = embedding_fn(query)
    negative_embedding = embedding_fn(negative)

    # Compute cosine similarity between query and negative embedding
    similarity = cosine_similarity([query_embedding], [negative_embedding])[0][0]
    # A lower similarity score indicates better rejection of negative answers
    return similarity

def calculate_counterfactual_robustness(query, fakeanswer, embedding_fn):
    """
    Calculate counterfactual robustness by measuring the cosine similarity between
    the query and the fake answer. A lower similarity means higher robustness.
    """
    query_embedding = embedding_fn(query)
    fakeanswer_embedding = embedding_fn(fakeanswer)

    # Compute cosine similarity between query and fakeanswer embedding
    similarity = cosine_similarity([query_embedding], [fakeanswer_embedding])[0][0]
    # A lower similarity score indicates better rejection of counterfactual information
    return similarity

def process_csv_and_calculate_metrics(csv_file_path):
    """
    Read the CSV file, calculate noise robustness, information integration,
    negative rejection, and counterfactual robustness for each row, and return the results.
    """
    results = []
    noise_robustness_vals = []
    integration_answer_vals = []
    integration_fakeanswer_vals = []
    integration_positive_vals = []
    integration_negative_vals = []
    negative_rejection_vals = []
    counterfactual_robustness_vals = []

    with open(csv_file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)

        for row in reader:
            query = row['query']
            answer = row['answer']
            #fakeanswer = row['fakeanswer']
            positive = row['positive']
            negative = row['negative']

            # Calculate Noise Robustness for the query
            noise_robustness = calculate_noise_robustness(query, get_embedding)
            noise_robustness_vals.append(noise_robustness)

            # Calculate Information Integration (compare query with answer, fakeanswer, positive, negative)
            integration_answer = calculate_information_integration(query, answer, get_embedding)
            #integration_fakeanswer = calculate_information_integration(query, fakeanswer, get_embedding)
            integration_positive = calculate_information_integration(query, positive, get_embedding)
            integration_negative = calculate_information_integration(query, negative, get_embedding)

            integration_answer_vals.append(integration_answer)
            #integration_fakeanswer_vals.append(integration_fakeanswer)
            integration_positive_vals.append(integration_positive)
            integration_negative_vals.append(integration_negative)

            # Calculate Negative Rejection (compare query with negative answer)
            negative_rejection = calculate_negative_rejection(query, negative, get_embedding)
            negative_rejection_vals.append(negative_rejection)

            # Calculate Counterfactual Robustness (compare query with fakeanswer)
            #counterfactual_robustness = calculate_counterfactual_robustness(query, fakeanswer, get_embedding)
            #counterfactual_robustness_vals.append(counterfactual_robustness)

            # Store the results in a list
            results.append({
                'id': row['id'],
                'query': query,
                'noise_robustness': noise_robustness,
                'integration_answer': integration_answer,
                #'integration_fakeanswer': integration_fakeanswer,
                'integration_positive': integration_positive,
                'integration_negative': integration_negative,
                'negative_rejection': negative_rejection
                #'counterfactual_robustness': counterfactual_robustness,
            })

    # Calculate averages for all metrics
    avg_noise_robustness = np.mean(noise_robustness_vals)
    avg_integration_answer = np.mean(integration_answer_vals)
    #avg_integration_fakeanswer = np.mean(integration_fakeanswer_vals)
    avg_integration_positive = np.mean(integration_positive_vals)
    avg_integration_negative = np.mean(integration_negative_vals)
    avg_negative_rejection = np.mean(negative_rejection_vals)
    avg_counterfactual_robustness = np.mean(counterfactual_robustness_vals)

    # Print the averages
    print("Average Noise Robustness:", avg_noise_robustness)
    print("Average Information Integration (Answer):", avg_integration_answer)
    #print("Average Information Integration (Fake Answer):", avg_integration_fakeanswer)
    print("Average Information Integration (Positive):", avg_integration_positive)
    print("Average Information Integration (Negative):", avg_integration_negative)
    print("Average Negative Rejection:", avg_negative_rejection)
    print("Average Counterfactual Robustness:", avg_counterfactual_robustness)

    return results

# Example usage
csv_file_path = '/content/outputrefine.csv'  # Update with the actual CSV file path
metrics = process_csv_and_calculate_metrics(csv_file_path)

# Print results for each row
for metric in metrics:
    print(metric)


In [ ]:
streamlit run "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py"

In [ ]:
pip install chromadb transformers

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer
import json

# Step 1: Load JSON data from a file
json_file_path = '/content/en_fact.json'

with open(json_file_path, 'r') as file:
    data = json.load(file)

# Step 2: Limit the data to the first 10 rows
data = data[:10]

# Step 3: Initialize the sentence transformer for embedding generation
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Step 4: Create a Chroma client and collection
client = chromadb.Client()
collection = client.create_collection("super_bowl_data23")

# Step 5: Prepare the data for encoding (query, answer, fakeanswer)
queries = [item['query'] for item in data]
answers = [item['answer'] for item in data]
fake_answers = [item['fakeanswer'] for item in data]

# Step 6: Encode the text into embeddings (for query, answer, and fakeanswer)
query_embeddings = model.encode(queries)
answer_embeddings = model.encode(answers)
fake_answer_embeddings = model.encode(fake_answers)

# Step 7: Insert the embeddings into ChromaDB collection (1 record per entry)
for idx, item in enumerate(data):
    # Add 3 separate records: one for each (query, answer, fakeanswer)
    # First, add the query
    collection.add(
        ids=[str(item['id']) + "_query"],  # Unique ID for query
        metadatas=[{'type': 'query', 'original_id': str(item['id'])}],  # Metadata for query
        embeddings=[query_embeddings[idx]],  # Store the query embedding
        documents=[item['query']]  # Store the query text
    )

    # Second, add the answer
    collection.add(
        ids=[str(item['id']) + "_answer"],  # Unique ID for answer
        metadatas=[{'type': 'answer', 'original_id': str(item['id'])}],  # Metadata for answer
        embeddings=[answer_embeddings[idx]],  # Store the answer embedding
        documents=[item['answer']]  # Store the answer text
    )

    # Third, add the fake answer
    collection.add(
        ids=[str(item['id']) + "_fake_answer"],  # Unique ID for fake answer
        metadatas=[{'type': 'fake_answer', 'original_id': str(item['id'])}],  # Metadata for fake answer
        embeddings=[fake_answer_embeddings[idx]],  # Store the fake answer embedding
        documents=[item['fakeanswer']]  # Store the fake answer text
    )

print("Data stored in ChromaDB.")



In [ ]:
pip install openai


In [ ]:
pip install chromadb


In [ ]:
import chromadb
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import random

# Initialize ChromaDB Client
client = chromadb.Client()
collection = client.get_collection("super_bowl_data23")

# Load the models for evaluation
def load_model(model_name):
    # This function assumes a HuggingFace model for simplicity.
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return tokenizer, model

# Load models (Replace with correct model names)
#gptj_tokenizer, gptj_model = load_model('EleutherAI/gpt-j-6B')  # Example GPT-J model
bloom_tokenizer, bloom_model = load_model('bigscience/bloom-560m')  # Example BLOOM model
#vicuna_tokenizer, vicuna_model = load_model('EleutherAI/gpt-neo-1.3B')  # GPT-Neo 1.3B model
#qwen_tokenizer, qwen_model = load_model('EleutherAI/gpt-j-6B')  # GPT-J 6B model

# Initialize the SentenceTransformer for text embedding
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def add_noise(embeddings, noise_ratio=0.1):
    noisy_embeddings = []
    for emb in embeddings:
        if isinstance(emb, np.ndarray):  # If embedding is an array
            noise = np.random.normal(0, noise_ratio, emb.shape)  # Adjust noise based on the shape
        else:  # If embedding is scalar
            noise = np.random.normal(0, noise_ratio)  # Generate scalar noise
        noisy_embeddings.append(emb + noise)
    return np.array(noisy_embeddings)

def calculate_metrics():
    noise_ratio = 0.1  # You can adjust the noise ratio here

    # Get the data from the ChromaDB collection
    results = collection.get(include=["metadatas", "documents", "embeddings"])

    for result in results['documents']:
        metadata = results['metadatas'][results['documents'].index(result)]
        query = result[0]
        real_answer = result[1]
        fake_answer = result[2]

        # Noise Robustness: Add noise to the query embeddings and check performance drop
        query_embedding = results['embeddings'][results['documents'].index(result)][0]
        noisy_query_embedding = add_noise([query_embedding], noise_ratio)

        # Tokenize the query (noisy version) before feeding it to the model
        noisy_query_text = query  # You can modify the noisy query if necessary
        input_ids = bloom_tokenizer(noisy_query_text, return_tensors="pt").input_ids

        # Now generate output based on token IDs, not embeddings
        bloom_output = bloom_model.generate(input_ids)

        print(f"Generated output (BLOOM): {bloom_output}")
        decoded_output = bloom_tokenizer.decode(bloom_output[0], skip_special_tokens=True)
        print("Generated text:", decoded_output)

# Step 4: Define the function for generating counterfactual data
def generate_counterfactual(data):
    # Modify data slightly to simulate counterfactual examples
    counterfactual_data = data.copy()
    for key in ['answer', 'fakeanswer']:
        counterfactual_data[key] = random.choice(['Tampa, Florida', 'Glendale, Arizona'])  # Simulate change
    return counterfactual_data

# Step 5: Define the function for Negative Rejection (Evaluating model's ability to reject fake answers)
def reject_fake_answer(model, tokenizer, query, real_answer, fake_answer):
    # Run both real and fake answers through the model
    input_real = tokenizer(query + " " + real_answer, return_tensors="pt")
    input_fake = tokenizer(query + " " + fake_answer, return_tensors="pt")

    # Generate output logits or scores for both inputs
    real_output = model(**input_real)
    fake_output = model(**input_fake)

    # We use the score or probability from the model to determine if it rejects fake answers
    real_score = torch.softmax(real_output.logits, dim=-1).max().item()
    fake_score = torch.softmax(fake_output.logits, dim=-1).max().item()

    # If real_score > fake_score, the model accepts the real answer and rejects the fake one
    rejection = real_score > fake_score
    return rejection, real_score, fake_score

# Step 7: Run the metrics calculation
calculate_metrics()



In [ ]:
import chromadb
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import random

# Initialize ChromaDB Client
client = chromadb.Client()
collection = client.get_collection("super_bowl_data23")

# Load the models for evaluation
def load_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return tokenizer, model

# Load models (BLOOM in this case)
bloom_tokenizer, bloom_model = load_model('bigscience/bloom-560m')

# Initialize the SentenceTransformer for text embedding
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def add_noise(embeddings, noise_ratio=0.1):
    noisy_embeddings = []
    for emb in embeddings:
        if isinstance(emb, np.ndarray):  # If embedding is an array
            noise = np.random.normal(0, noise_ratio, emb.shape)  # Adjust noise based on the shape
        else:  # If embedding is scalar
            noise = np.random.normal(0, noise_ratio)  # Generate scalar noise
        noisy_embeddings.append(emb + noise)
    return np.array(noisy_embeddings)

def calculate_metrics():
    noise_ratio = 0.1  # You can adjust the noise ratio here

    # Get the data from the ChromaDB collection
    results = collection.get(include=["metadatas", "documents", "embeddings"])

    for result in results['documents']:
        metadata = results['metadatas'][results['documents'].index(result)]
        query = result[0]
        real_answer = result[1]
        fake_answer = result[2]

        # Noise Robustness: Add noise to the query embeddings and check performance drop
        query_embedding = results['embeddings'][results['documents'].index(result)][0]
        noisy_query_embedding = add_noise([query_embedding], noise_ratio)

        # Tokenize the query (noisy version) before feeding it to the model
        noisy_query_text = query  # You can modify the noisy query if necessary
        input_ids = bloom_tokenizer(noisy_query_text, return_tensors="pt").input_ids

        # Generate output for noisy input
        bloom_output = bloom_model.generate(input_ids)
        decoded_output_noisy = bloom_tokenizer.decode(bloom_output[0], skip_special_tokens=True)

        print(f"Original query: {query}")
        print(f"Noisy query generated output: {decoded_output_noisy}")

        # Calculate BLEU or other similarity metrics between original and noisy output
        bleu_score = calculate_bleu_score(decoded_output_noisy, query)
        print(f"Noise Robustness (BLEU): {bleu_score}")

        # Step 5: Negative Rejection
        rejection, real_score, fake_score = reject_fake_answer(bloom_model, bloom_tokenizer, query, real_answer, fake_answer)
        print(f"Negative Rejection: {rejection} (Real score: {real_score}, Fake score: {fake_score})")

        # Step 6: Information Integration (comparison of noisy vs clean output)
        clean_input_ids = bloom_tokenizer(query, return_tensors="pt").input_ids
        clean_output = bloom_model.generate(clean_input_ids)
        decoded_output_clean = bloom_tokenizer.decode(clean_output[0], skip_special_tokens=True)

        # Compare noisy vs clean outputs
        bleu_score_clean = calculate_bleu_score(decoded_output_noisy, decoded_output_clean)
        print(f"Information Integration (BLEU): {bleu_score_clean}")

        # Step 7: Counterfactual Robustness
        counterfactual_data = generate_counterfactual({'answer': real_answer, 'fakeanswer': fake_answer})
        counterfactual_query = query.replace(real_answer, counterfactual_data['answer'])

        counterfactual_input_ids = bloom_tokenizer(counterfactual_query, return_tensors="pt").input_ids
        counterfactual_output = bloom_model.generate(counterfactual_input_ids)
        decoded_output_counterfactual = bloom_tokenizer.decode(counterfactual_output[0], skip_special_tokens=True)

        # Compare counterfactual output with the original
        counterfactual_similarity = calculate_bleu_score(decoded_output_clean, decoded_output_counterfactual)
        print(f"Counterfactual Robustness (BLEU): {counterfactual_similarity}")

# Function to calculate BLEU score
def calculate_bleu_score(output1, output2):
    from nltk.translate.bleu_score import sentence_bleu
    reference = output2.split()
    hypothesis = output1.split()
    return sentence_bleu([reference], hypothesis)

# Function to calculate Negative Rejection (Real vs Fake)
def reject_fake_answer(model, tokenizer, query, real_answer, fake_answer):
    input_real = tokenizer(query + " " + real_answer, return_tensors="pt")
    input_fake = tokenizer(query + " " + fake_answer, return_tensors="pt")

    # Generate output logits or scores for both inputs
    real_output = model(**input_real)
    fake_output = model(**input_fake)

    # We use the score or probability from the model to determine if it rejects fake answers
    real_score = torch.softmax(real_output.logits, dim=-1).max().item()
    fake_score = torch.softmax(fake_output.logits, dim=-1).max().item()

    # If real_score > fake_score, the model accepts the real answer and rejects the fake one
    rejection = real_score > fake_score
    return rejection, real_score, fake_score

# Step 7: Run the metrics calculation
calculate_metrics()


In [ ]:
import chromadb
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import random
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu

# Initialize ChromaDB Client
client = chromadb.Client()
collection = client.get_collection("super_bowl_data23")

# Function to load models from HuggingFace
def load_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return tokenizer, model

# Load models (different models here)
models = {
    "BLOOM": load_model('bigscience/bloom-560m'),
    "GPT-Neo": load_model('EleutherAI/gpt-neo-2.7B'),  # GPT-Neo 2.7B model
    "Qwen": load_model('qwen/qwen-7b'),
    "MiniLLM": load_model('sentence-transformers/all-MiniLM-L6-v2')
}

# Initialize the SentenceTransformer for text embedding
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Add noise to embeddings
def add_noise(embeddings, noise_ratio=0.1):
    noisy_embeddings = []
    for emb in embeddings:
        if isinstance(emb, np.ndarray):  # If embedding is an array
            noise = np.random.normal(0, noise_ratio, emb.shape)  # Adjust noise based on the shape
        else:  # If embedding is scalar
            noise = np.random.normal(0, noise_ratio)  # Generate scalar noise
        noisy_embeddings.append(emb + noise)
    return np.array(noisy_embeddings)

# Calculate BLEU score
def calculate_bleu_score(output1, output2):
    reference = output2.split()
    hypothesis = output1.split()
    return sentence_bleu([reference], hypothesis)

# Calculate Negative Rejection (Real vs Fake)
def reject_fake_answer(model, tokenizer, query, real_answer, fake_answer):
    input_real = tokenizer(query + " " + real_answer, return_tensors="pt")
    input_fake = tokenizer(query + " " + fake_answer, return_tensors="pt")

    # Generate output logits or scores for both inputs
    real_output = model(**input_real)
    fake_output = model(**input_fake)

    # We use the score or probability from the model to determine if it rejects fake answers
    real_score = torch.softmax(real_output.logits, dim=-1).max().item()
    fake_score = torch.softmax(fake_output.logits, dim=-1).max().item()

    # If real_score > fake_score, the model accepts the real answer and rejects the fake one
    rejection = real_score > fake_score
    return rejection, real_score, fake_score

# Function to calculate the metrics for each model
def calculate_metrics():
    noise_ratio = 0.1  # You can adjust the noise ratio here

    # Get the data from the ChromaDB collection
    results = collection.get(include=["metadatas", "documents", "embeddings"])

    metrics = []

    for result in results['documents']:
        metadata = results['metadatas'][results['documents'].index(result)]
        query = result[0]
        real_answer = result[1]
        fake_answer = result[2]

        # Loop over all models and calculate metrics
        for model_name, (tokenizer, model) in models.items():
            # Noise Robustness: Add noise to the query embeddings and check performance drop
            query_embedding = results['embeddings'][results['documents'].index(result)][0]
            noisy_query_embedding = add_noise([query_embedding], noise_ratio)

            # Tokenize the query (noisy version) before feeding it to the model
            noisy_query_text = query  # You can modify the noisy query if necessary
            input_ids = tokenizer(noisy_query_text, return_tensors="pt").input_ids

            # Generate output for noisy input
            bloom_output = model.generate(input_ids)
            decoded_output_noisy = tokenizer.decode(bloom_output[0], skip_special_tokens=True)

            # Noise Robustness
            bleu_score = calculate_bleu_score(decoded_output_noisy, query)
            noise_robustness_percentage = bleu_score * 100

            # Negative Rejection
            rejection, real_score, fake_score = reject_fake_answer(model, tokenizer, query, real_answer, fake_answer)
            negative_rejection_percentage = (real_score - fake_score) * 100

            # Information Integration: Compare noisy vs clean output
            clean_input_ids = tokenizer(query, return_tensors="pt").input_ids
            clean_output = model.generate(clean_input_ids)
            decoded_output_clean = tokenizer.decode(clean_output[0], skip_special_tokens=True)

            # Compare noisy vs clean outputs
            bleu_score_clean = calculate_bleu_score(decoded_output_noisy, decoded_output_clean)
            information_integration_percentage = bleu_score_clean * 100

            # Counterfactual Robustness
            counterfactual_data = generate_counterfactual({'answer': real_answer, 'fakeanswer': fake_answer})
            counterfactual_query = query.replace(real_answer, counterfactual_data['answer'])

            counterfactual_input_ids = tokenizer(counterfactual_query, return_tensors="pt").input_ids
            counterfactual_output = model.generate(counterfactual_input_ids)
            decoded_output_counterfactual = tokenizer.decode(counterfactual_output[0], skip_special_tokens=True)

            # Compare counterfactual output with the original
            counterfactual_similarity = calculate_bleu_score(decoded_output_clean, decoded_output_counterfactual)
            counterfactual_robustness_percentage = counterfactual_similarity * 100

            # Store results for this query
            metrics.append({
                'Model': model_name,
                'Noise Robustness': noise_robustness_percentage,
                'Negative Rejection': negative_rejection_percentage,
                'Information Integration': information_integration_percentage,
                'Counterfactual Robustness': counterfactual_robustness_percentage
            })

    # Convert metrics to a pandas DataFrame
    df = pd.DataFrame(metrics)

    # Calculate averages for each metric
    averages = df.mean()

    # Add averages row to the dataframe
    averages_row = averages.append(pd.Series({'Model': 'Average'}))
    df = df.append(averages_row, ignore_index=True)

    # Display the results in a tabular format
    print(df)

# Function to generate counterfactual data (this is a placeholder, implement as needed)
def generate_counterfactual(data):
    # Placeholder for counterfactual generation logic
    return {'answer': data['fakeanswer']}  # For example, swap the answers

# Step 7: Run the metrics calculation
calculate_metrics()
